In [25]:
import json

In [3]:
with open(r"train.json","r") as read_file:
          train = json.load(read_file)

In [4]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?',
    'answers': [{'text': 'Brandon Sanderson', 'answer_start': 71}]}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?',
    'answers': [{'text': 'between 2006 and 2008', 'answer_start': 28}]},
   {'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?',
    'answers': [{'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages',
      'answer_start': 63}]},
   {'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?',
    'answers': []}]}]

In [5]:
with open(r"test.json","r") as read_file:
          test= json.load(read_file)

In [6]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?',
    'answers': [{'text': 'region called the Final Empire', 'answer_start': 38},
     {'text': 'world called Scadrial', 'answer_start': 74}]}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?',
    'answers': [{'text': 'one', 'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?',
    'answers': []}]}]

In [7]:
import logging 
from simpletransformers.question_answering import  QuestionAnsweringModel,QuestionAnsweringArgs

c:\Users\karti\Desktop\qna\qna\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_type="bert"
model_name="bert_base_cased"
if model_type=="bert":
    model_name="bert-base-cased"
elif model_type=="roberta":
    model_name="roberta-base"
elif model_type=="distilbert":
    model_name="distilbert-base-cased"
elif model_type=="distilroberta":
    model_type="roberta"
    model_name="distilroberta-base"
elif model_type=="electra-base":
    model_type="electra"
    model_name="google/electra-base-discriminator"
elif model_type=="electra-small":
    model_type="electra"
    model_name="xlnet-base-cased"





In [9]:
#Configure the model
model_args=QuestionAnsweringArgs()
model_args.train_batch_size=16
model_args.evaluate_during_training=True
model_args.n_best_size=3
model_args.num_train_epochs=5


#advanced methodology


In [18]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [19]:
#initialize the model
model=QuestionAnsweringModel(
    model_type,model_name,args=train_args
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# train th model
model.train_model(train,eval_data=test)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\karti\Desktop\qna\qna\Lib\site-packages\simpletransformers\question_answering\question_answering_model.py:697: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Epoch 1 of 10:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\karti\Desktop\qna\qna\Lib\site-packages\simpletransformers\question_answering\question_answering_model.py:720: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
c:\Users\karti\Desktop\qna\qna\Lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the lear

(10,
 {'global_step': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'correct': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
  'similar': [2, 3, 2, 2, 2, 2, 1, 1, 1, 1],
  'incorrect': [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
  'train_loss': [4.823567867279053,
   4.7760419845581055,
   4.2864580154418945,
   3.81640625,
   3.3408203125,
   2.8395180702209473,
   2.6748046875,
   2.3426105976104736,
   1.9241535663604736,
   1.961669921875],
  'eval_loss': [-0.40869140625,
   -0.49267578125,
   -0.5771484375,
   -0.6533203125,
   -0.72119140625,
   -0.78173828125,
   -0.8330078125,
   -0.873046875,
   -0.90087890625,
   -0.9150390625]})

In [21]:
#evaluate the model
result,texts=model.eval_model(test)

Running Evaluation: 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


In [22]:
result

{'correct': 1, 'similar': 1, 'incorrect': 1, 'eval_loss': -0.9150390625}

In [23]:
#make prediction with the model 
to_predict=[
    {
        "context": "Vin is a Mistborn of great power and skills.",
        "qas":[
            {
                "question": "What is Vin's speciality?",
                "id":"0",
            }
        ],
    }
]

In [24]:
answers,probabilities=model.predict(to_predict)
print(answers)

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\karti\Desktop\qna\qna\Lib\site-packages\simpletransformers\question_answering\question_answering_model.py:1348: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  4.43it/s]

[{'id': '0', 'answer': ['and skills']}]
